In [8]:

import matplotlib as plt
plt.style.use('dark_background')


# imports

import tensorflow as tf

import insightface
from insightface.app import FaceAnalysis
from mtcnn import MTCNN
from PIL import Image
import cv2
from tensorflow.keras.models import load_model


import numpy as np
from numpy import dot
from numpy.linalg import norm


# extra imports

import psycopg2
import matplotlib.pyplot as plt
from google.colab import files
import os





AttributeError: module 'matplotlib' has no attribute 'style'

In [ ]:
# load models

app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0) # load arcface/insight  and detects faces, so no need for mtcnn


#detector = MTCNN() # detect faces

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
# # railway connect

# db_info = {
#     "host": "hopper.proxy.rlwy.net",
#     "port": 55475,
#     "dbname": "railway",
#     "user": "postgres",
#     "password": "syDhOVUIPvVxDdAlEztnmJdpvMZqxPUt"
# }

# try:
#     conn = psycopg2.connect(**db_info)
#     cursor = conn.cursor()
#     print("✅ Connected to Railway PostgreSQL")
# except Exception as e:
#     print("❌ Connection failed:", e)

✅ Connected to Railway PostgreSQL


In [ ]:
# postgres login

db_info = {
    "host": "localhost",
    "port": "5432",
    "dbname": "face_embeddings",
    "user": "postgres",
    "password": "Ottawa2006!?"
}

# create table

create_table_sql = """
CREATE TABLE IF NOT EXISTS embeddings (
    id SERIAL PRIMARY KEY,
    person_name VARCHAR(100),
    embedding FLOAT8[]  -- Stores the face embedding as an array
);
"""

try:
    conn = psycopg2.connect(**db_info)
    cursor = conn.cursor()
    cursor.execute(create_table_sql)
    conn.commit()
    cursor.close()
    conn.close()
    print("✅ Table 'embeddings' created successfully.")
except Exception as e:
    print("❌ Error:", e)


# insert embeddings

def insert_embedding(name, embedding):
    conn = psycopg2.connect(**db_info)
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO embeddings (person_name, embedding) VALUES (%s, %s)",
        (name, embedding.tolist())
    )
    conn.commit()
    cursor.close()
    conn.close()


In [ ]:
def get_embedding(image_path):
    img = np.array(Image.open(image_path).convert('RGB'))
    faces = app.get(img)
    if not faces:
        print(f"❌ No face detected in {image_path}")
        return None
    return faces[0].embedding


In [ ]:
# insert embeddings as JSON
def insert_embedding(person_name, embedding, db_info):
    try:
        conn = psycopg2.connect(**db_info)
        cursor = conn.cursor()

        # Convert numpy array to JSON string
        embedding_json = json.dumps(embedding.tolist())

        cursor.execute("""
            INSERT INTO face_embeddingsss (person_name, embeddings)
            VALUES (%s, %s);
        """, (person_name, embedding_json))

        conn.commit()
        cursor.close()
        conn.close()
        print("✅ Embedding inserted successfully for:", person_name)

    except Exception as e:
        print("❌ Failed to insert embedding:", e)

# Usage
insert_embedding("person1", get_embedding("person1.webp"), db_info)
insert_embedding("person2", get_embedding("person2.webp"), db_info)


✅ Embedding inserted successfully for: person1
✅ Embedding inserted successfully for: person2


In [ ]:
def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

face1_emb = get_embedding(file1)
face2_emb = get_embedding(file2)

if face1_emb is None or face2_emb is None:
    print("❌ One or both images couldn't be read or no face detected.")
else:
    similarity = cosine_similarity(face1_emb, face2_emb)
    print(f"Similarity: {similarity:.3f}")
    if similarity > 0.6:
        print("✅ Same person")
    else:
        print("❌ Different people")


Similarity: 0.081
❌ Different people


In [ ]:
# import matplotlib.pyplot as plt

# face1 = crop_face('person1.webp')
# face2 = crop_face('person2.webp')

# if face1 is not None and face2 is not None:
#     plt.figure(figsize=(6,3))
#     plt.subplot(1,2,1)
#     plt.imshow(face1)
#     plt.title("person1")

#     plt.subplot(1,2,2)
#     plt.imshow(face2)
#     plt.title("person2")

#     plt.show()
# else:
#     print("❌ Failed to detect one or both faces.")
